In [18]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from pylab import rcParams

In [19]:
filename='sm_bidi2.h5'
dim = 15

import pickle
with open('tk.pickle', 'rb') as handle:
    tk = pickle.load(handle)

In [20]:
from keras.models import load_model

autoencoder = load_model(filename)

In [21]:
unph = pd.read_csv('devph.csv')
ph = unph.values.flatten().tolist()
len(ph)

4181

In [22]:
num_words = len(ph)
maxlen = 30
embed_dim = 150
batch_size = 16
nb_epoch = 30

In [23]:
with open("./dev-br.txt") as f:
    sentences = f.readlines()


In [24]:
print(len(sentences))
test_set = sentences[205000:]
print(len(test_set))

207180
2180


In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = num_words, split=' ')
tokenizer.fit_on_texts(sentences)
seqs = tokenizer.texts_to_sequences(test_set)
pad_seqs = pad_sequences(seqs, maxlen)

In [26]:
#get the MSE error term
predictions = autoencoder.predict(pad_seqs)
#mse = np.mean(np.power(scaled_seqs - predictions, 2), axis=1)
#print('MSE:', np.quantile(mse, 0.999)) # => the 9999% quatile - only 0.0001% have error score higher than that

2021-08-02 08:30:15.883898: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1093749600 exceeds 10% of free system memory.


In [65]:
#classes = list(np.argmax(predictions, axis=1))
#print(predictions)
sentence = "पी त मा ञ्जि ष्ठ व र्णा श"
seq = tokenizer.texts_to_sequences([sentence])
pad_seq = pad_sequences(seq, maxlen)
sentence_seq = autoencoder.predict(pad_seq)
classes = list(np.argmax(sentence_seq[0], axis=1))
print(classes)
print(sentence)
print(tokenizer.sequences_to_texts([classes]))
#for i in range(len(sentence_seq(0)):
   
print(pad_seq[0])
#sentence_seq.shape
mse = np.mean(np.power(pad_seq - classes, 2), axis=1)
print('MSE:', np.quantile(mse, 0.999)) # => the 9999% quatile - only 0.0001% have error score higher than that

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 216, 5, 18, 1163, 209, 7, 259, 8]
पी त मा ञ्जि ष्ठ व र्णा श
['पी त मा ञ्जि ष्ठ व र्णा श']
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0  216    5   18 1163  209    7
  259    8]
MSE: 0.0


In [84]:
#encode all the data
encoded_seqs = encode_sequence_list(seqs_ds.iloc[:,0],dim)
#scale it
scaled_data = MinMaxScaler().fit_transform(encoded_seqs)
#predict it
predicted = autoencoder.predict(scaled_data)
#get the error term
mse = np.mean(np.power(scaled_data - predicted, 2), axis=1)
#now add them to our data frame
seqs_ds['MSE'] = mse

In [85]:
display(seqs_ds)

,0,MSE,MSE_Outlier
0,sevyatām,0.006087,1
1,ditisutāḥ,0.004921,0
2,thomas,0.001726,0


In [83]:
mse_threshold = np.quantile(seqs_ds['MSE'], 0.9999)
print(f'MSE 0.9999 threshhold:{mse_threshold}')
seqs_ds['MSE_Outlier'] = 0
seqs_ds.loc[seqs_ds['MSE'] > mse_threshold, 'MSE_Outlier'] = 1
print(f"Num of MSE outlier:{seqs_ds['MSE_Outlier'].sum()}")

seqs_ds.loc[seqs_ds['MSE_Outlier'] == 1]

MSE 0.9999 threshhold:0.006086682941338582
Num of MSE outlier:1


,0,MSE,MSE_Outlier
0,sevyatām,0.006087,1
